# Problème 2 : Equilibre de dissociation du CO2 à H et P constants

#### numéro d'équipe : 
#### Nom, prénom et matricule des membres:
####    -
####    -
####    -

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
from math import*

from thermoprop import *
from solveur import *

### Fonctions fournies pour la résolution du problème (à compléter)

In [ ]:
def calc_LHS(gas, species, T):
    """
    Renvoie le LHS avec la constantes d'équilibre Kp associée à la réaction calculée avec les enthalpies libres
    
    gas : objet cantera représentant le mélange étudié
    species : liste des strings des espèces dans le mélange
    T : température du mélange
    """ 
    R = ct.gas_constant*1e-3       # Constante universelle des gaz parfaits [J/mol/K]
    P0 = ct.one_atm                # Pression de référence [Pa]
    
    # Calcul des enthalpie libre de chacune des espèces dans l'équation en [J/mol]
    g = {}
    for sp in species:
        g[sp] = calc_thermoprop_mix(gas, [sp], [1], T, P0)[3]
    
    # Calcul de la constante d'équilibre de la réaction de dissociation
    Kp = # à remplir
    
    return Kp

def calc_RHS(alpha, species, P):
    """
    Renvoie le RHS avec la constantes d'équilibre Kp associée à la réaction calculée avec la composition du mélange
    
    alpha : quantité de CO2 dissociée
    species : liste des strings des espèces dans le mélange
    P : pression dans le mélange
    """
    P0 = ct.one_atm                # Pression de référence [Pa]
    
    # Calcul du RHS de l'équation d'équilibre chimique (alpha)
    RHS = # à remplir
    
    return RHS

def calc_comp_TP_cst(alpha_est, gas, species, T, P):
    """
    Renvoie la quantité de CO2 dissociée à T, P constants à l'équilibre du mélange
    
    alpha_est : estimation initiale de la quantité de CO2 dissociée
    gas : objet cantera représentant le mélange étudié
    species : liste des strings des espèces dans le mélange
    T : température du mélange
    P : pression dans le mélange
    """
    # Paramètres pour le solveur
    epsi_alpha = 1e-8                     # Tolérance relative
    max_iter = 1000                       # Nombre maximal d'itération pour le solveur

    # Calcul du left hand side (LHS) de l'équation d'équilibre chimique (avec l'enthalpie libre)
    LHS = calc_LHS(gas, species, T)       # fonction à compléter
    
    # Appel du solveur pour trouver la composition chimique à T, P constants
    def fun_eq(X):
        # Calcul du right hand side (LHS) de l'équation d'équilibre chimique (avec l'enthalpie libre)
        RHS = calc_RHS(X, species, P)     # fonction à compléter
        return RHS-LHS
    alpha_eq = solveur_Newton_Raphson(fun_eq, alpha_est, abs(LHS)*epsi_alpha, max_iter)
    
    return alpha_eq

def calc_H_TP_cst(T, alpha_est, gas, species, P, alpha_flag):
    """
    Renvoie l'enthalpie des produits
    
    T : température du mélange
    alpha_est : estimation initiale de la quantité de CO2 dissociée (ou quantité de CO2 dissociée si alpha_flag=1)
    gas : objet cantera représentant le mélange étudié
    species : liste des strings des espèces dans le mélange
    P : pression dans le mélange
    alpha_flag : flag pour réaliser ou non l'équilibre chimique à T, P constant
    """
    if alpha_flag:
        # Calcul de la quantité de CO2 dissociée à T,P constant
        alpha = calc_comp_TP_cst(alpha_est, gas, species, T, P)
    else:
        alpha = alpha_est
    
    # Quantité de matière des produits
    N_P = # à remplir
    
    # Enthalpie des produits
    H_P = calc_thermoprop_mix(gas, species, N_P, T, P)[1]
    
    return H_P

## 2.a Méthode de la constante d'équilibre Kp

### Code principal

In [ ]:
%%time
gas=ct.Solution("mech_pb1-2.yaml")
gas.basis="molar"

# Paramètres du problème
T0 = 2500                                           # Température initiale [K]
P = 10*ct.one_atm                                   # Pression [Pa]

# Définition des réactifs
species = ["CO", "O2", "CO2"]                            # Liste des espèces dans le mélange
N_R = [0, 0, 1]                                          # Quantité de matière associée à chaque espèce dans les réactifs
H_R = calc_thermoprop_mix(gas, species, N_R, T0, P)[1]   # Enthalpie des réactifs

# Paramètres pour le solveur
alpha_est = 0.1                   # Estimation initiale pour la quantité dissociée de CO2
T_est = 2400                      # Estimation initiale pour la température
epsi_T = 1e-6                     # Tolérance relative
max_iter = 10                     # Nombre maximal d'itération pour le solveur

# Calcul de la température des produits à l'équilibre
def fun_T(T):
    H_P = calc_H_TP_cst(T, alpha_est, gas, species, P, 1)
    return H_P-H_R
T_eq = solveur_Newton_Raphson(fun_T, T_est, abs(H_R)*epsi_T, max_iter)

# Calcul de la composition des produits à la température adiabatique de flamme
alpha_eq = #à remplir

print("Solveur perso : A l'équilibre, la quantité de CO2 dissociée est de {:.4f} à la température de {:.0f}K".format(alpha_eq, T_eq))

### Comparaison avec Cantera

In [ ]:
%%time
gas = ct.Solution('mech_pb1-2.yaml')
gas.basis="molar"

T0 = 2500
P = 10*ct.one_atm
species = ["CO", "O2", "CO2"]
N = [0, 0, 1]

gas.TPX = T0, P, "CO2:1"
gas.equilibrate("HP")

alpha_cant = gas.X[gas.species_index("CO")]
n_tot = 1 + alpha_cant/2
T_cant = gas.T

print("Cantera : A l'équilibre, la quantité de CO2 dissociée est de {:.4f} à la température de {:.0f}K".format(alpha_cant*n_tot, T_cant))

## 2.b Maximisation de l'entropie

### Code principal

In [ ]:
%%time
gas = ct.Solution('mech_pb1-2.yaml')
gas.basis="molar"

# Paramètres du problème
T0 = 2500                                           # Température initiale [K]
P = 10*ct.one_atm                                   # Pression [Pa]
alpha_arr = np.arange(0., 0.30001, 0.0001)

# Définition des réactifs
species = ["CO", "O2", "CO2"]                            # Liste des espèces dans le mélange
N_R = [0, 0, 1]                                          # Quantité de matière associée à chaque espèce dans les réactifs
H_R = calc_thermoprop_mix(gas, species, N_R, T0, P)[1]   # Enthalpie des réactifs

# arrays pour l'affichage
u_arr = np.zeros(alpha_arr.shape)
h_arr = np.zeros(alpha_arr.shape)
s_arr = np.zeros(alpha_arr.shape)
g_arr = np.zeros(alpha_arr.shape)
T_arr = np.zeros(alpha_arr.shape)

# Paramètres pour le solveur
T_est = 2400                      # Estimation initiale pour la température
epsi_T = 1e-6                     # Tolérance relative
max_iter = 10                     # Nombre maximal d'itération pour le solveur

# Timers pour évaluation des performances
for i_a, alpha in enumerate(alpha_arr):
    # Liste des quantités de matière du mélange
    N_P = #à remplir
    
    # Calcul de la température des produits pour la alpha correspondant
    def fun_T(T):
        H_P = calc_H_TP_cst(T, alpha, gas, species, P, 0)
        return H_P-H_R
    T_P = solveur_Newton_Raphson(fun_T, T_est, abs(H_R)*epsi_T, max_iter)
    
    # Récupération de la température du mélange
    T_arr[i_a] = T_P                           # array pour l'affichage
    T_est = T_P                                # actualisation de l'estimation de température pour la prochaine itération
    
    # Récupération des propriétés thermodynamique du mélange
    [u_arr[i_a], h_arr[i_a], s_arr[i_a], g_arr[i_a]] = calc_thermoprop_mix(gas, species, N_P, T_P, P)
    
alpha_calc = #à remplir
T_calc = #à remplir

print("Maximisation de l'entropie : la quantité de CO2 dissocié est {:.4f} à la température de {:.0f}K".format(alpha_calc, T_calc))

### Affichage

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(10,10), dpi=100)

axs[0,0].plot(alpha_arr, u_arr*1e-3)
axs[0,0].set_ylabel("u [kJ/mol de CO2 initiale]")
axs[0,0].set_xlabel(r"quantité dissociée $\alpha$")
axs[0,0].axvline(alpha_cant, color="red", linestyle="--")

axs[0,1].plot(alpha_arr, h_arr*1e-3)
axs[0,1].set_ylabel("h [kJ/mol de CO2 initiale]")
axs[0,1].set_xlabel(r"quantité dissociée $\alpha$")
axs[0,1].set_ylim(-500, 0)
axs[0,1].axvline(alpha_cant, color="red", linestyle="--")

axs[1,0].plot(alpha_arr, g_arr*1e-3)
axs[1,0].set_ylabel("g [kJ/mol de CO2 initiale]")
axs[1,0].set_xlabel(r"quantité dissociée $\alpha$")
axs[1,0].axvline(alpha_cant, color="red", linestyle="--")

axs[1,1].plot(alpha_arr, s_arr*1e-3)
axs[1,1].set_ylabel("s [kJ/mol de CO2 initiale/K]")
axs[1,1].set_xlabel(r"quantité dissociée $\alpha$")
axs[1,1].axvline(alpha_cant, color="red", linestyle="--")

axs[2,0].plot(alpha_arr, T_arr)
axs[2,0].set_ylabel("T [K]")
axs[2,0].set_xlabel(r"quantité dissociée $\alpha$")
axs[2,0].axvline(alpha_cant, color="red", linestyle="--")

fig.suptitle("Problème 2\n Maximisation de l'entropie")
fig.tight_layout()
plt.show()